In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
!git clone https://github.com/sugyan/gan-playground.git

Cloning into 'gan-playground'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 194 (delta 118), reused 114 (delta 52), pack-reused 0
Receiving objects: 100% (194/194), 39.61 KiB | 9.90 MiB/s, done.
Resolving deltas: 100% (118/118), done.


### SavedModel from StyleGAN2-ADA network-pkl

See https://github.com/sugyan/stylegan2-ada/blob/cpu/pkl2savedmodel.py

In [ ]:
# !curl -L https://github.com/sugyan/gan-playground/releases/download/2021-03-09/idol-face-256.tar.gz -o savedmodel.tar.gz
!curl -L https://github.com/sugyan/gan-playground/releases/download/2021-03-09/idol-face-512.tar.gz -o savedmodel.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   628  100   628    0     0   3900      0 --:--:-- --:--:-- --:--:--  3900
100  101M  100  101M    0     0  42.3M      0  0:00:02  0:00:02 --:--:-- 54.2M


In [ ]:
!tar zxvf savedmodel.tar.gz

./savedmodel/
./savedmodel/variables/
./savedmodel/saved_model.pb


In [ ]:
%mkdir /content/images
%mkdir /content/results
%cd /content/gan-playground

/content/gan-playground


In [ ]:
import pathlib

import cv2
import numpy as np
import pandas as pd

# Generate images

In [ ]:
import generate


savedmodel = pathlib.Path("/content/savedmodel")
seed = 1
num_images = 20000
outdir = pathlib.Path("/content/images")
save_dlatents = True

generate.main(savedmodel, seed, num_images, outdir, save_dlatents)

# Analyze attributes

In [ ]:
%cd /content/gan-playground/analyze_attributes/

/content/gan-playground/analyze_attributes


## Emotion

In [ ]:
!pip install pypaz

     |████████████████████████████████| 92kB 6.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=b565e54139e110116012bcb92d02f9f897b8b1adc51a2ceb3e4257b2d8a88a53
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [ ]:
import emotion

results = emotion.predict(pathlib.Path("/content/images"))
np.savez("/content/results/emotions.npz", **results)

## Headpose

In [ ]:
!curl -O http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.0M  100 61.0M    0     0  1121k      0  0:00:55  0:00:55 --:--:-- 6024k


In [ ]:
import headpose

results = headpose.predict(pathlib.Path("/content/images"))
np.savez("/content/results/headposes.npz", **results)

## Parsing

In [ ]:
%cd /content/
!git clone https://github.com/shaoanlu/face_toolbox_keras.git

/content
Cloning into 'face_toolbox_keras'...
remote: Enumerating objects: 273, done.
remote: Total 273 (delta 0), reused 0 (delta 0), pack-reused 273
Receiving objects: 100% (273/273), 245.92 MiB | 34.28 MiB/s, done.
Resolving deltas: 100% (128/128), done.
Checking out files: 100% (42/42), done.


In [ ]:
%cd /content/face_toolbox_keras/

/content/face_toolbox_keras


In [ ]:
from typing import Dict

from models.parser import face_parser


def predict(target_dir: pathlib.Path) -> Dict[str, np.ndarray]:
    fp = face_parser.FaceParser()
    # fd = face_detector.FaceAlignmentDetector()
    # fp.set_detector(fd)
    results = {}
    for i, image_path in enumerate(map(str, target_dir.glob("*.png"))):
        print(f"{i:05} {image_path}")
        im = cv2.imread(image_path)[..., ::-1]
        results[image_path] = fp.parse_face(im)[0]

    return results


results = predict(pathlib.Path("/content/images"))
np.savez('/content/results/parsing_results.npz', **results)

In [ ]:
%cd /content/gan-playground/analyze_attributes/

/content/gan-playground/analyze_attributes


In [ ]:
import hair

results = hair.calculate(pathlib.Path("/content/results/parsing_results.npz"))
np.savez("/content/results/hairs.npz", **results)

# Calculate vectors

In [ ]:
from calc_vectors import calculate_vectors, load_emotion, load_hair, load_headpose


emotion_npz = pathlib.Path("/content/results/emotions.npz")
headpose_npz = pathlib.Path("/content/results/headposes.npz")
hair_npz = pathlib.Path("/content/results/hairs.npz")

rate = 0.005
out_file = pathlib.Path("/content/results/vectors.npz")

df = pd.DataFrame()
df = df.join(load_emotion(emotion_npz), how="outer")
df = df.join(load_hair(hair_npz), how="outer")
df = df.join(load_headpose(headpose_npz), how="outer")

results = calculate_vectors(df, rate)
np.savez(
    out_file.resolve(),
    **results,
    mean=np.mean([np.load(f"{index}.npy") for index in df.index], axis=0),
)

In [ ]:
from google.colab import files
files.download("/content/results/vectors.npz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>